In [1]:
import cvxpy
import random
import numpy as np
import pandas as pd

In [2]:
# サンプルデータの読み込み
sample = pd.read_csv('sample.csv')

In [3]:
sample.head(3)

,id,price_A,price_B,price_C,rating_A,rating_B,rating_C
0,1,380,385,389,4.1,3.3,4.0
1,2,943,947,961,10.1,8.9,9.3
2,3,980,980,987,10.4,8.7,10.3


In [4]:
# クライアトa, b, cの元々持っていた枠をランダムに決める。
length = len(sample)
ids = list(range(length))
# idは1からなので、1を足す。
random_ids = [id+1 for id in ids]

# Break Point 1の略。
bp1 = length//3 
bp2 = length//3 + bp1

random.shuffle(random_ids)
a, b, c = random_ids[:bp1], random_ids[bp1:bp2], random_ids[bp2:]

In [5]:
# それぞれのidを表示
print(a)
print(b)
print(c)

[14, 19, 29, 23, 5, 15, 26, 10, 8, 12]
[2, 18, 20, 16, 28, 24, 22, 21, 11, 6]
[9, 30, 7, 4, 25, 13, 1, 17, 27, 3]


In [6]:
# それぞれのidのデータのみを取り出すための判別用カラム。
sample['judge_a'] = sample['id'].apply(lambda x:x in a)
sample['judge_b'] = sample['id'].apply(lambda x:x in b)
sample['judge_c'] = sample['id'].apply(lambda x:x in c)

In [7]:
# それぞれのidのデータを取得。
df_a = sample[sample['judge_a']].loc[:, ['id','price_A', 'rating_A']]
df_b = sample[sample['judge_b']].loc[:, ['id','price_B', 'rating_B']]
df_c = sample[sample['judge_c']].loc[:, ['id','price_C', 'rating_C']]

In [8]:
# 判別用カラムを削除。
del sample['judge_a']
del sample['judge_b']
del sample['judge_c']

In [34]:
# 元々の値段の総和、元々の視聴率の和を計算する。
# Original price, Original point
original_price_A, original_point_A = sum(df_a['price_A']), sum(df_a['rating_A'])
original_price_B, original_point_B = sum(df_b['price_B']), sum(df_b['rating_B'])
original_price_C, original_point_C = sum(df_c['price_C']), sum(df_c['rating_C'])

In [35]:
# 獲得した枠のidを貯めておく。
obtained_id_A = []
obtained_id_B = []
obtained_id_C = []

In [36]:
# 獲得した枠の価格を貯めていく。
obtained_price_A = 0
obtained_price_B = 0
obtained_price_C = 0

In [37]:
# 獲得した枠の視聴率を貯めていく。
obtained_point_A = 0
obtained_point_B = 0
obtained_point_C = 0

In [38]:
# ここに結果を格納していき、何度も繰り返す。（視聴率和の合計値が'key'で、その時のidが'value'）
result = dict()

In [39]:
# 各クライアントの考える価値を数値化し、表に加える。
# 指定した視聴率/そのクライアントの号数価格
sample['value_A'] = round((sample['rating_A']/sample['price_A'])*1000, 2)
sample['value_B'] = round((sample['rating_B']/sample['price_B'])*1000, 2)
sample['value_C'] = round((sample['rating_C']/sample['price_C'])*1000, 2)

In [40]:
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)
sample.head(3)

client_Aの交換前の合計価格: 6278 client_Aの交換前の合計視聴率 62.5
client_Bの交換前の合計価格: 6042 client_Bの交換前の合計視聴率 61.199999999999996
client_Cの交換前の合計価格: 6682 client_Cの交換前の合計視聴率 65.6


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,2,943,947,961,10.1,8.9,9.3,10.71,9.40,9.68
2,3,980,980,987,10.4,8.7,10.3,10.61,8.88,10.44


<b><font color='Red'>※valueは、基本的に正の数じゃないと、選ばなくなってしまう（０の方が大きいから。）</font></b>

***
***

### 〜ここからアルゴリズム〜

In [51]:
# ナップザック問題。

# size = 値段のデータ。
# weight = 価値のデータ。
# capacity = 元々の値段×1.05倍の値段。
# ids = 取得すべきだと考えられた枠のid    

def Knapsack(size, weight, capacity, obtained=0, rate=0.05):
    # 価格が元の価格の1.05%を超えても、終わり。
    if capacity*(1 + rate) - obtained > 0:        
        x = cvxpy.Variable(size.shape[0], boolean=True)
        # 要素の個数(これを加えて平均を最大化しないと、価格の小さいものばかりをとってしまう。)
        count = sum(np.ones(len(size)) * x)
        # 目的
        objective = cvxpy.Maximize(weight * x / count)
        # 制限。ここでは、capacity(価格)の上下5%を取っている。
        constraints = [capacity*(1 + rate) -  obtained >= size * x]
        constraints += [capacity*(1 - rate) - obtained <= size * x]

        prob = cvxpy.Problem(objective, constraints)
        prob.solve(solver=cvxpy.ECOS_BB)
        result = [round(ix, 0) for ix in x.value]

        return result
    else:
        return []
    
# 2重のリストをフラットにする関数(重複は残る！)
def Flatten_dual(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

In [76]:
# namesには、'A', 'B', 'C', ... を入れるイメージ。

def Loop(df, *names):
    result = dict()
    num = len(names)
    original_prices = [original_price_A, original_price_B, original_price_C]
    original_points = [original_point_A, original_point_B, original_point_C]

    obtained_prices = [0 for i in range(num)]
    obtained_points = [0 for i in range(num)]

    obtained_idses = [[] for i in range(num)]
    add_idses = [[] for i in range(num)]
    idses = [[] for i in range(num)]
    check_point = 0
    
    while check_point==0:
        for i in range(num):
            original_price = original_prices[i]
            original_point = original_points[i]

            name = names[i]
            size = np.array(df['price_' + name])
            weight = np.array(df['value_' + name])
            capacity = original_price

            if obtained_points[i] < original_point:
                idses[i] = Knapsack(size, weight, capacity,  obtained_prices[i], rate=0.1)
            else:
                idses[i] = [0 for i in range(len(size))]

        ids = np.array(df['id'].index) + 1

        # それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
        ids_dict = dict()
        for i in ids-1:
            values = []
            for j in range(num):
                if i in idses[j] * ids -1:
                    values.append(names[j])
            if values:
                value = random.choice(values)
                ids_dict[i] = value

        for i in range(num):
            add_idses[i] = [ids for ids, client in ids_dict.items() if client == names[i]]
            obtained_idses[i] += add_idses[i]
            obtained_prices[i] += sum(df.query('index in ' + str(add_idses[i]))['price_' + name])
            obtained_points[i] += sum(df.query('index in ' + str(add_idses[i]))['rating_' + name])

        extraction_ids = Flatten_dual(add_idses)
        df = df.query('index not in ' + str(extraction_ids)).reset_index(drop=True)
        
        if len(df) == 1 and sample['id'][0] != 0:
            # np.zeros()の中の数字は、列の数。
            add_df = pd.DataFrame(np.zeros(df.shape[1])).T
            add_df.columns = df.columns
            df = pd.concat([df, add_df]).reset_index(drop=True)

        # 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
        if len(df) == 1:
            check_point += 1
        
        print('loop!!')
    
    if obtained_points > original_points:
        sum_point = sum(obtained_points)
        result[sum_point] = [obtained_ids]
                                  
    return result

In [85]:
Loop(sample, 'A', 'B', 'C')

loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!
loop!!


KeyboardInterrupt: 

### これを何度も繰り返すことにより、resultに値を格納しまくる。

In [125]:
# 最も視聴率和を大きくした交換の仕方を求める。
result[max(result.keys())]

[[1, 2, 16, 28, 0, 3, 9, 0, 2, 3, 4],
 [4, 3, 5, 7, 8, 13, 20, 4, 11, 0],
 [13, 27, 4, 15, 12, 14, 6, 7, 10]]

### プロトタイプは完成。あとはエラーの処理をし、アルゴリズム化する。